<h3>Necessary packages and libraries</h3>

In [1]:
!pip install psycopg2-binary neo4j pandas pymongo redis

import psycopg2
import pandas as pd
from neo4j import GraphDatabase
import pymongo
import redis
import json

<h3>Database Connection and Check</h3>

In [2]:
# PostgreSQL (change to your own database/user/password/host/port)
pg_conn = psycopg2.connect(
    database="amazon_db",
    user="postgres",
    password="101123",
    host="localhost",
    port="5432"
)
pg_cursor = pg_conn.cursor()

# Test PostgreSQL
pg_cursor.execute("SELECT COUNT(*) FROM amazon_products;")
result = pg_cursor.fetchone()
print("PostgreSQL is connected, total products:", result[0])

# Neo4j
neo4j_driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "20001011")) # Change to your own

# Test Neo4j 
def test_neo4j_connection():
    query = "MATCH (p:Product) RETURN count(p) AS total_products"
    with neo4j_driver.session() as session:
        result = session.run(query)
        total_products = result.single()["total_products"]
        print("Neo4j is connected, total products:", total_products)

test_neo4j_connection()

# MongoDB
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
db = mongo_client["amazon"]
promoted_collection = db["promoted_products"]

# Redis 
r = redis.Redis(host='localhost', port=6379, db=0)

PostgreSQL is connected, total products: 548552
Neo4j is connected, total products: 548552


<h3>Helper function for fetching</h3>

In [3]:
def fetch_product_details(pg_cursor, asins):
    if not asins:
        return []
    query = """
    SELECT 
        p.asin, 
        p.title, 
        p.product_group, 
        p.sales_rank,
        COALESCE(AVG(r.rating), 0) AS avg_rating
    FROM 
        amazon_products p
    LEFT JOIN 
        amazon_reviews r ON p.asin = r.asin
    WHERE 
        p.asin IN %s
    GROUP BY 
        p.asin, p.title, p.product_group, p.sales_rank
    ORDER BY 
        p.sales_rank ASC
    """
    pg_cursor.execute(query, (tuple(asins),))
    return pg_cursor.fetchall()

<h3>Main Recommendation Function</h3>

In [4]:
def get_recommendations(search_input, search_by_asin, pg_cursor, neo4j_driver):
    # Define function to fetch promoted products
    def get_promoted_products():
        try:
            # Check Redis cache first
            cached_data = r.keys("promoted:*")
            if cached_data:
                promoted = [json.loads(r.get(k).decode()) for k in cached_data]
                return promoted[:3]
            print("Fetching from MongoDB...")
            # Fetch from MongoDB if not in cache
            promoted = list(promoted_collection.find().limit(3))
            # Cache in Redis with 7-day expiration
            for p in promoted:
                p["_id"] = str(p["_id"])
                r.set(f"promoted:{p['asin']}", json.dumps(p), ex=604800)  # 604800 seconds = 7 days
            return promoted
        except Exception as e:
            print(f"Error fetching promoted products: {e}")
            return []

    # Generate main recommendations
    if not search_by_asin:
        # Title Search
        # Step 1: Find matching ASINs
        matching_query = """
        SELECT asin FROM amazon_products 
        WHERE title ILIKE %s 
        ORDER BY sales_rank ASC 
        LIMIT 3
        """
        pg_cursor.execute(matching_query, (f"%{search_input}%",))
        matching_asins = [row[0] for row in pg_cursor.fetchall()]

        if not matching_asins:
            return "⚠ No matching products found."

        # Step 2: Get recommendation ASINs from Neo4j
        neo4j_query = """
        MATCH (p:Product)-[:CO_PURCHASED_WITH]->(p2)
        WHERE p.asin IN $matching_asins AND p.louvainCommunity = p2.louvainCommunity
        RETURN DISTINCT p2.asin AS recommended_asin
        """
        with neo4j_driver.session() as session:
            result = session.run(neo4j_query, matching_asins=matching_asins)
            neo4j_recommendations = [record["recommended_asin"] for record in result 
                                    if record["recommended_asin"] not in matching_asins]

        # Step 3: Fetch all products in one query
        all_asins = matching_asins + neo4j_recommendations
        all_products = fetch_product_details(pg_cursor, all_asins)

        # Step 4: Split results into matching and recommendations
        matching_products = [p for p in all_products if p[0] in matching_asins]
        recommendations_products = [p for p in all_products if p[0] not in matching_asins][:7]

        # Step 5: Create main recommendations DataFrame
        main_products = matching_products + recommendations_products
        main_df = pd.DataFrame(main_products, 
                              columns=["ASIN", "Title", "Product Group", "Sales Rank", "Avg Rating"])
        main_df['is_promoted'] = False

    else:
        # ASIN Search
        # Step 1: Verify the ASIN exists
        product_query = """
        SELECT asin FROM amazon_products WHERE asin = %s
        """
        pg_cursor.execute(product_query, (search_input,))
        product_asin = pg_cursor.fetchone()
        if not product_asin:
            return "⚠ Product not found."

        # Step 2: Get recommendation ASINs from Neo4j
        neo4j_query = """
        MATCH (p:Product {asin: $asin})-[:CO_PURCHASED_WITH]->(p2)
        WHERE p.louvainCommunity = p2.louvainCommunity
        RETURN DISTINCT p2.asin AS recommended_asin
        """
        with neo4j_driver.session() as session:
            result = session.run(neo4j_query, asin=search_input)
            neo4j_recommendations = [record["recommended_asin"] for record in result 
                                    if record["recommended_asin"] != search_input]

        # Step 3: Fetch all products in one query
        all_asins = [search_input] + neo4j_recommendations
        all_products = fetch_product_details(pg_cursor, all_asins)

        # Step 4: Split into specific product and recommendations
        product = next(p for p in all_products if p[0] == search_input)
        recommendations_products = [p for p in all_products if p[0] != search_input][:9]

        # Step 5: Create main recommendations DataFrame
        main_products = [product] + recommendations_products
        main_df = pd.DataFrame(main_products, 
                              columns=["ASIN", "Title", "Product Group", "Sales Rank", "Avg Rating"])
        main_df['is_promoted'] = False

    # Add promoted products
    promoted = get_promoted_products()
    promoted_asins = [p['asin'] for p in promoted]
    promoted_details = fetch_product_details(pg_cursor, promoted_asins)
    promoted_df = pd.DataFrame(promoted_details, 
                              columns=["ASIN", "Title", "Product Group", "Sales Rank", "Avg Rating"])
    promoted_df['is_promoted'] = True

    # Combine main recommendations and promoted products
    final_df = pd.concat([main_df, promoted_df], ignore_index=True)

    print("✅ Recommendations generated with promoted products!")
    return final_df

<h3>Manually Modified Promoted Products</h3>

In [5]:
def add_promoted_product(asin, promotion_reason):
    # Fetch product details from PostgreSQL
    pg_cursor.execute("""
        SELECT asin, title, product_group, sales_rank
        FROM amazon_products
        WHERE asin = %s
    """, (asin,))
    product = pg_cursor.fetchone()
    if not product:
        print(f"❌ Product {asin} not found!")
        return
    
    # Prepare data for MongoDB insertion
    promoted_data = {
        "asin": product[0],
        "title": product[1],
        "reason": promotion_reason,
        "added_date": "2025-03-18"  # Static date for demo
    }
    
    # Insert into MongoDB
    promoted_collection.insert_one(promoted_data)
    
    # Prepare cache data without _id
    cache_data = {
        "asin": product[0],
        "title": product[1],
        "reason": promotion_reason,
        "added_date": "2025-03-18"
    }
    
    # Cache in Redis
    r.set(f"promoted:{product[0]}", json.dumps(cache_data), ex=604800)  # 7 days
    print(f"✅ Promoted product added: {product[0]}")
# Clear existing promoted products (optional, for demo consistency)
promoted_collection.delete_many({})
r.flushdb()

# Manually add three promoted products
add_promoted_product("0738700797", "Featured in Spring Sale")
add_promoted_product("1559362022", "Wake Up and Smell the Coffee")
add_promoted_product("078510870X", "Ultimate Marvel Team-Up")

✅ Promoted product added: 0738700797
✅ Promoted product added: 1559362022
✅ Promoted product added: 078510870X


<h3>Demo</h3>

In [7]:
# Demo: Title Search for "the great gatsby"
get_recommendations("the great gatsby", False, pg_cursor, neo4j_driver)


✅ Recommendations generated with promoted products!


,ASIN,Title,Product Group,Sales Rank,Avg Rating,is_promoted
0,0684801523,The Great Gatsby,Book,956,4.1809421841541756,False
1,6301247485,The Great Gatsby,Video,2851,3.8000000000000000,False
2,0764586017,Fitzgerald's The Great Gatsby (Cliffs Notes),Book,4163,3.0000000000000000,False
3,0316769487,The Catcher in the Rye,Book,60,4.1683593750000000,False
4,0142000663,The Grapes of Wrath : (Centennial Edition),Book,201,4.2920696324951644,False
5,0553210092,The Scarlet Letter,Book,368,3.5143678160919540,False
6,0553210793,The Adventures of Huckleberry Finn (Bantam Cla...,Book,1138,3.9966555183946488,False
7,B00007KQA4,Of Mice And Men (Special Edition),DVD,4915,4.5760869565217391,False
8,0764586041,The Adventures of Huckleberry Finn (Cliffs Notes),Book,9206,4.3333333333333333,False
9,0764585886,The Crucible (Cliffs Notes),Book,18300,3.5454545454545455,False


In [8]:
get_recommendations("0684801523", True, pg_cursor, neo4j_driver)

✅ Recommendations generated with promoted products!


,ASIN,Title,Product Group,Sales Rank,Avg Rating,is_promoted
0,0684801523,The Great Gatsby,Book,956,4.1809421841541756,False
1,0316769487,The Catcher in the Rye,Book,60,4.1683593750000000,False
2,0142000663,The Grapes of Wrath : (Centennial Edition),Book,201,4.2920696324951644,False
3,0553210092,The Scarlet Letter,Book,368,3.5143678160919540,False
4,0553210793,The Adventures of Huckleberry Finn (Bantam Cla...,Book,1138,3.9966555183946488,False
5,0764586017,Fitzgerald's The Great Gatsby (Cliffs Notes),Book,4163,3.0000000000000000,False
6,0738700797,Candlemas: Feast of Flames,Book,168596,4.3333333333333333,True
7,1559362022,Wake Up and Smell the Coffee,Book,518927,3.7500000000000000,True
8,078510870X,Ultimate Marvel Team-Up,Book,612475,3.6250000000000000,True


In [9]:
get_recommendations("Boat Racing", False, pg_cursor, neo4j_driver)

✅ Recommendations generated with promoted products!


,ASIN,Title,Product Group,Sales Rank,Avg Rating,is_promoted
0,0070308179,Around the Buoys: A Manual of Sailboat Racing ...,Book,134829,3.5000000000000000,False
1,0393308014,The Tactics of Small Boat Racing,Book,151367,4.6666666666666667,False
2,1898660379,Keelboat & Sportsboat Racing,Book,960464,0,False
3,0805003517,"Sailing Smart : Winning Techniques, Tactics, A...",Book,27752,5.0000000000000000,False
4,0393032965,A Manual of Sail Trim,Book,61567,3.7142857142857143,False
5,0393303330,Advanced Racing Tactics,Book,151015,3.6666666666666667,False
6,0312042787,Championship Tactics : How Anyone Can Sail Fas...,Book,247295,4.2500000000000000,False
7,0924486813,The New Book of Sail Trim,Book,311419,2.6666666666666667,False
8,0071376097,Paul Elvstrom Explains the Racing Rules of Sai...,Book,368845,4.5000000000000000,False
9,0738700797,Candlemas: Feast of Flames,Book,168596,4.3333333333333333,True
